In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import thanos
from thanos.gen import *
from thanos.util import *
from thanos.operator import *

Notes: examples that look more like ITOM data

## Constant and Random Variables

In [ ]:
a = ConstantVariable(2.0)
X = RandomVariable('gamma', seeded_gen(1), 1.0, 2.0)

for _ in range(10):
    print(a(), X())

## Random Seed Management

In [ ]:
b = ConstantVariable(X) # A constant variable from a gamma distribution

# variables as iterator generators
for _, sample in zip(range(3), b):
    print(sample)

In [ ]:
b.reseed(seeded_gen(123))
for _, sample in zip(range(3), b):
    print(sample)

## Composability

In [ ]:
seed = seeded_gen(123)
Y = op(operator.add, 
       ConstantVariable(RandomVariable('uniform', seed, 0.0, 10.0)), 
       RandomVariable('gamma', seed, 1.0, 2.0))

In [ ]:
plot_examplesappyl, 3)

## Repeatability

In [ ]:
Y.reset()
plot_examples(Y, 2)

## Operator Support

In [ ]:
plot_examples(Y.reset() * 2 + 10)

## Generation Tree

In [ ]:
print_optree(Y *2 + 10)

## Fluent Interface (Pandas-like)

In [ ]:
Y_smooth = Y.reset().window(60, sliding=True).apply(np.mean)
plot_examples(Y_smooth, 2)

## Streaming With Pytorch

In [ ]:
# make this more visual by adding different kinds of series, then injecting the anomalies
%matplotlib inline
from thanos.SyntheticDataset import *
ds = SyntheticDataset(Y_smooth.reset(), seeded_gen(5678))
animate_plot(ds)

# Simple Signals

## Seasonality

In [ ]:
seasonal = op(operator.add,
              RandomVariable('gamma', seed, 1.0, 1.5), 
              Seasonality() * ConstantVariable(RandomVariable('uniform', seed, 0.0, 3.0)))
plot_examples(seasonal)

## Trend

In [ ]:
trended = op(operator.add, seasonal, Trend(rate= 1/(60*24)))
plot_examples(trended)

## Business Hours

In [ ]:
daily_smooth = uniop(Seasonality(period = (17-9)*60, phase=today_offset()), np.abs)
business = op(mix, Conditional(DateModulation(this_week()), daily_smooth, ConstantVariable(0.0)), 
              RandomVariable('uniform', seed, 0.0, 3.0)
             )
plot_examples(business)

# Basic Use Cases

## Test Harness for Model Correctness

This is an example of testing a model from sklearn, to verify that it can learn the parameters of a normal distribution
based on synthetic data generated by THANOS

In [ ]:
import pytest
import ipytest

ipytest.autoconfig()

In [ ]:
%%ipytest -qq -s
seed = seeded_gen(np.random.randint(0, 2**32))

@pytest.mark.repeat(1)
@pytest.mark.parametrize("random_variable", [RandomVariable("normal", seed, ConstantVariable(RandomVariable("uniform", seed, 0, 10)), 1.0)])
@pytest.mark.parametrize("sample_count", [1000, 10000, 100000])
def test_GMM_fit(random_variable,sample_count):
    import sklearn.mixture
    gmm = sklearn.mixture.GaussianMixture()
    a = np.fromiter(random_variable.reseed(seed), float, count=sample_count)
    r = gmm.fit(a[:, np.newaxis])
    assert np.isclose(r.means_[0,0], random_variable.params[0](), rtol=1e-1)
    print("fit mean : %f, var : %f, true mean: %f" % (r.means_[0, 0], r.covariances_[0, 0], random_variable.params[0]()))

## Anomaly Injection

In [ ]:
anomalies = BinMarkovAnomaly(seed, p_on = 0.001, p_off = 0.05) + ConstantVariable(3.0)
plot_examples(anomalies, 3)

### Mixing it all together

In [ ]:
composed_example = ReBasis(seeded_gen(123), np.array([[0.7, 0.2, 0.1], [0.7, 0.1, 0.2], [0.3, 0.4, 0.3], [0.1, 0.4, 0.5]]), [BinMarkovAnomaly(seeded_gen(123), p_on = 0.0001, p_off = 0.03), Y_smooth, trended])
ds = SyntheticDataset(composed_example, seeded_gen(5678))
animate_plot(ds, batch_size = 128, batch_count = 30)

In [ ]:
print(ds.anomalies())

# Advanced Signal Examples

These examples are from the public service now documentation of signal classification as documented in [ITOM-Health](https://docs.servicenow.com/bundle/quebec-it-operations-management/page/product/event-management/concept/operational-intelligence-overview.html)

## Positive Clipped Noisy

In [ ]:
positive_clipped_noisy = RandomVariable('gamma', seed, 2.0, 3.0).apply(np.clip, 10, np.inf)
plot_examples(positive_clipped_noisy)

## Centered Noisy (bounded)

In [ ]:
plot_examples(RandomVariable('uniform', seed, 2.0, 7.0))

# Skewed Noisy (bounded, biased)

In [ ]:
plot_examples(RandomVariable('gamma', seed, 2.0, 3.0).apply(np.clip, 4, 18))

## Accumulator

In [ ]:
accumulator = Trend(rate=1, initial_value=ConstantVariable(RandomVariable('uniform', seed, 0, 2000)), maximum=ConstantVariable(RandomVariable('uniform', seed, 1500, 2000)), minimum=20)
plot_examples(accumulator)

## MultiNomial

In [ ]:
statecount = 4
plot_examples(StateSelector(RandomVariable('gamma', seed, 1, 2)*10,
                            RandomVariable('integers', seed, 0, statecount),
                            [ConstantVariable(RandomVariable('uniform', seed, 0, 100)) for i in range(statecount) ]),
              3)

# Model Uncertainty As Training Progresses